In [64]:
import numpy as np
from pyntcloud import PyntCloud
import open3d as o3d
import matplotlib.pyplot as plt
import cv2

In [65]:
print(o3d.__version__)

0.18.0


In [66]:
def read_point_cloud(file_or_cloud):
    if isinstance(file_or_cloud, PyntCloud):
        return file_or_cloud
    elif isinstance(file_or_cloud, str):
        cloud = PyntCloud.from_file(file_or_cloud)
        return cloud
    else:
        raise ValueError("Input must be a file path (str) or a Point Cloud object.")


In [67]:
def pyntcloud_to_open3d(file_or_cloud):
    cloud = read_point_cloud(file_or_cloud)
    cloud_pts = cloud.points[['x', 'y', 'z', 'intensity']].values

    xyz = cloud_pts[:, :3]
    open3d_cloud = o3d.geometry.PointCloud()
    open3d_cloud.points = o3d.utility.Vector3dVector(xyz)
    return open3d_cloud

In [68]:
def color_pcd(pcd, clr='r'):
    # cloud = PreprocessorUtils.read_point_cloud(file)
    # pcd = PlaygroundUtils.pyntcloud_to_open3d(cloud)

    if clr == 'r':
        color = [1, 0, 0]
        pcd.paint_uniform_color(color)
    elif clr == 'g':
        color = [0, 1, 0]
        pcd.paint_uniform_color(color)
    elif clr == 'b':
        color = [0, 0, 1]
        pcd.paint_uniform_color(color)

    return pcd

In [69]:
def visualize_multiple_pcds(*pcds, normal=False):
    clouds = [pcd for pcd in pcds]
    o3d.visualization.draw_geometries(clouds, point_show_normal=normal)

In [70]:
def visualize_clouds(inlier_cloud, outlier_cloud):
    inlier_pcd = pyntcloud_to_open3d(inlier_cloud)
    outlier_pcd = pyntcloud_to_open3d(outlier_cloud)
    
    inlier_pcd = color_pcd(inlier_pcd, 'r')
#     outlier_pcd = color_pcd(outlier_pcd, 'b')
    
    visualize_multiple_pcds(inlier_pcd, outlier_pcd)

In [71]:
def detect_edges_by_normals(pcd, radius, threshold):
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=5))

    normals = np.asarray(pcd.normals)
    edges = np.zeros(len(normals), dtype=bool)

    kdtree = o3d.geometry.KDTreeFlann(pcd)

    for i in range(len(normals)):
        [_, idx, _] = kdtree.search_radius_vector_3d(pcd.points[i], radius)
        neighbor_normals = normals[idx, :]
        norm_var = np.mean(np.linalg.norm(neighbor_normals - normals[i], axis=1))

        if norm_var > threshold:
            edges[i] = True

    edge_points = pcd.select_by_index(np.where(edges)[0])

    return edge_points

## Ground Map Logic implementation

In [185]:
ply_frame = '/home/maanz/Downloads/DGV/LidarPipeline_runs/hamburg/maps/ground_map.ply'

In [194]:
VS = 0.1
RADIUS = 1
NBS = 20
ANG = 10

In [195]:
def voxelize_cloud(pcd, vs):
    downpcd = pcd.voxel_down_sample(voxel_size=vs)
    return downpcd

In [196]:
def estimate_normals(pcd, radius, max_nbs):
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, 
                                                          max_nn=max_nbs))

    normals = np.asarray(pcd.normals)
    return pcd, normals

In [198]:
def filter_by_normals(pcd, normals, angle_deg):
    # Calculate the cosine of the angle for comparison
    cos_angle = np.cos(np.radians(angle_deg))

    # Determine the range for exclusion based on cosine values
    vertical_mask = np.abs(normals[:, 2]) >= cos_angle
    inlier_indices = np.where(~vertical_mask)[0]
    outlier_indices = np.where(vertical_mask)[0]

    # Extract inlier and outlier point clouds
    inlier_pcd = pcd.select_by_index(inlier_indices)
    outlier_pcd = pcd.select_by_index(outlier_indices)

    return inlier_pcd, outlier_pcd

In [199]:
def flow(ply_frame, voxelize = True):
    pcd = pyntcloud_to_open3d(ply_frame)
    if voxelize:
        pcd = voxelize_cloud(pcd, VS)
    pcd, normals = estimate_normals(pcd, RADIUS, NBS)
    INLIER, OUTLIER = filter_by_normals(pcd, normals, ANG)
    
    return INLIER, OUTLIER


In [200]:
Inlier, Outlier = flow(ply_frame, True)

In [202]:
Inlier = color_pcd(Inlier, 'r')
# visualize_multiple_pcds(Inlier, True)
o3d.visualization.draw_geometries([Inlier],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                 point_show_normal=False)

## Single Frame Processing

In [197]:
def segment_plane(pcd, dist_thresh, ransac_n, num_iters):
    # Apply plane segmentation
    plane_model, inliers = pcd.segment_plane(distance_threshold=dist_thresh,
                                             ransac_n=ransac_n,
                                             num_iterations=num_iters)
    
    inlier_pcd = pcd.select_by_index(inliers)
    outlier_pcd = pcd.select_by_index(inliers, invert=True)
    
    return inlier_pcd, outlier_pcd

In [213]:
input_frame = '/home/maanz/Downloads/DGV/LidarPipeline_runs/hamburg/preprocessed/frames/1693307439225051803.ply'
input_pcd = pyntcloud_to_open3d(input_frame)

In [225]:
# visualize_multiple_pcds(input_pcd)

In [231]:
inlier, outlier = segment_plane(input_pcd, 0.1, 6, 1000)

In [232]:
inlier = color_pcd(inlier, 'r')
visualize_multiple_pcds(inlier, outlier)

In [235]:
inlier, normals = estimate_normals(inlier, 0.05, 3)

In [ ]:
o3d.visualization.draw_geometries([inlier],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                 point_show_normal=True)